Copyright 2024 shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Vertex AI Search - REST API 호출로 검색

Feedback : shins777@gmail.com

이 Colab 은 대부분의 고객의 요구사항이면서, 많은 유즈케이스에서의 근간이 되는 지식검색에 대한 예제입니다.
Gronding service 로 Vertex AI Search와 Langchain 을 연동해서 정보를 검색하고 해당 검색된 정보를 Gemini 를 통해서 추론해서 답해주는 형태의 예제입니다.

* Vertex AI Search Manual references
    * https://cloud.google.com/enterprise-search?hl=ko
    * https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search


이 소스에서 지식 검색의 대상은 "정보통신 진흥 및 융합 활성화 등에 관한 특별법" 이며 해당 특별법은 아래 사이트에서 다운받았습니다.
https://www.law.go.kr/

#라이브러리 설치
*   Langchain library : https://github.com/langchain-ai/langchain
*   Langchain Vertex AI API : https://api.python.langchain.com/en/stable/google_vertexai_api_reference.html



In [12]:
%pip install --upgrade --quiet langchain langchain-core langchain-google-vertexai google-cloud-discoveryengine

Note: you may need to restart the kernel to use updated packages.


In [13]:
from IPython.display import display, Markdown

### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [14]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  
* 현재 Gemini는 한국리전(asia-northeast3)을 통해서 접근이 가능합니다.
* 아래 SEARCH_URL Vertex AI Search를 구성 후 Integration 메뉴항목에서 추출된 값입니다. 

In [15]:
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"
MODEL = "gemini-1.0-pro-001"
SEARCH_URL = "https://discoveryengine.googleapis.com/v1alpha/projects/721521243942/locations/global/collections/default_collection/dataStores/it-laws-ds_1713063479348/servingConfigs/default_search:search"

In [16]:
import vertexai
import google
import google.oauth2.credentials
from google.auth import compute_engine
import google.auth.transport.requests
import requests
import json
import os

vertexai.init(project=PROJECT_ID, location=REGION)

stream = os.popen('gcloud auth print-access-token')
credential_token = stream.read().strip()


### Vertex AI Search 검색

아래 코드는 REST API를 통해서 Verext AI Search에 직접 검색 쿼리를 보내는 예제입니다.
검색에 활용되는 패러미터는 ContentSearchSpec 에 따릅니다. 아래 정보 참고하세요.
* https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1beta.types.SearchRequest.ContentSearchSpec

특히 ExtractiveContentSpec은 검색 컨텐츠를 좀더 상세하게 얻을수 있는 설정입니다. 아래 정보 참고하세요.
* https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1beta.types.SearchRequest.ContentSearchSpec.ExtractiveContentSpec

In [17]:
def retrieve_vertex_ai_search(question:str, search_url:str, page_size:int)->str:

  """ retrieve information from enterprise search ( discovery engine )"""

  # Create a credentials token to call a REST API
  headers = {
      "Authorization": "Bearer "+ credential_token,
      "Content-Type": "application/json"
  }

  query_dic ={
      "query": question,
      "page_size": str(page_size),
      "offset": 0,
      "contentSearchSpec":{

            "searchResultMode" : "CHUNKS",
            "chunkSpec" : {
                "numPreviousChunks" : 2,
                "numNextChunks" : 2
            }              
      },
  }

  data = json.dumps(query_dic)

  # Encode data as UTF8
  data=data.encode("utf8")

  response = requests.post(search_url,headers=headers, data=data)

  print(response.text)
  return response.text


#### Search 결과 파싱 로직
아래 로직은 검색된 결과를 파싱하는 로직입니다.

In [18]:
def parse_chunks(response_text:str)->dict:

    """Parse response to build a conext to be sent to LLM"""

    dict_results = json.loads(response_text)

    index = 0
    search_results = {}

    if dict_results.get('results'):

        for result in dict_results['results']:

            item = {}

            chunk = result['chunk']

            #print(chunk)
            # print(f"content {chunk['content']}")

            # print(f"documentMetadata {chunk['documentMetadata']}")
            # print(f"documentMetadata {chunk['documentMetadata']['title']}")
            # print(f"documentMetadata {chunk['documentMetadata']['uri']}")

            # print(f"documentMetadata {chunk['pageSpan']['pageStart']}")
            # print(f"documentMetadata {chunk['pageSpan']['pageEnd']}")

            # print(f"previousChunks {chunk['chunkMetadata']['previousChunks'][0]['content']}")
            # print(f"nextChunks {chunk['chunkMetadata']['nextChunks'][0]['content']}")

            item['title'] = chunk['documentMetadata']['title']
            item['uri'] = chunk['documentMetadata']['uri']
            item['pageSpan'] = f"{chunk['pageSpan']['pageStart']} ~ {chunk['pageSpan']['pageEnd']}"
            item['content'] = chunk['content']

            # chunk 는 현재 Contents에 가까운것 부터 나타남.
            p_chunks = chunk['chunkMetadata']['previousChunks']
            for p_chunk in p_chunks:
                item['content'] = p_chunk['content'] +"\n"+ item['content']

            n_chunks = chunk['chunkMetadata']['nextChunks']
            for n_chunk in n_chunks:
                item['content'] = item['content'] +"\n"+ n_chunk['content']

            search_results[f'results-{index}'] = item
            index = index+1

    return search_results

In [19]:
question = "개인정보 보호법에 대해서 설명해주세요."

page_size = 2

searched_ctx = retrieve_vertex_ai_search(question, SEARCH_URL, page_size)


{
  "results": [
    {
      "chunk": {
        "name": "projects/721521243942/locations/global/collections/default_collection/dataStores/it-laws-ds_1713063479348/branches/0/documents/1/chunks/c32",
        "id": "c32",
        "content": "\n# 제7장 개인정보 분쟁조정위원회 \u003c개정 2020. 2. 4.\u003e\n제56조(확정판결의 효력) 원고의 청구를 기각하는 판결이 확정된 경우 이와 동일한 사안에 관하여는 제51조에 따른 다 른 단체는 단체소송을 제기할 수 없다. 다만, 다음 각 호의 어느 하나에 해당하는 경우에는 그러하지 아니하다. 1. 판결이 확정된 후 그 사안과 관련하여 국가·지방자치단체 또는 국가·지방자치단체가 설립한 기관에 의하여 새 로운 증거가 나타난 경우 2. 기각판결이 원고의 고의로 인한 것임이 밝혀진 경우 제57조(「민사소송법」의 적용 등) ① 단체소송에 관하여 이 법에 특별한 규정이 없는 경우에는 「민사소송법」을 적용한 다. ② 제55조에 따른 단체소송의 허가결정이 있는 경우에는 「민사집행법」제4편에 따른 보전처분을 할 수 있다. ③ 단체소송의 절차에 관하여 필요한 사항은 대법원규칙으로 정한다. 제9장 보칙 \u003c개정 2020. 2. 4.\u003e 제58조(적용의 일부 제외) ① 다음 각 호의 어느 하나에 해당하는 개인정보에 관하여는 제3장부터 제8장까지를 적용하 지 아니한다. \u003c개정 2023. 3. 14.\u003e 1. 삭제 \u003c2023. 3. 14.\u003e 2. 국가안전보장과 관련된 정보 분석을 목적으로 수집 또는 제공 요청되는 개인정보 3. 삭제 \u003c2023. 3. 14.\u003e 4. 언론, 종교단체, 정당이 각각 취재·보도, 선교, 선거 입후보자 추천 등 고유 목적을 달성하기 위하여 수집ㆍ이용 

In [20]:
context = parse_chunks(searched_ctx)

print(context)

{'results-0': {'title': '개인정보 보호법(법률)(제19234호)(20240315)', 'uri': 'gs://it_laws_kr/law_pdf/개인정보 보호법(법률)(제19234호)(20240315).pdf', 'pageSpan': '27 ~ 41', 'content': '\n# 제7장 개인정보 분쟁조정위원회 <개정 2020. 2. 4.>\n제40조(설치 및 구성) ① 개인정보에 관한 분쟁의 조정(調停)을 위하여 개인정보 분쟁조정위원회(이하 “분쟁조정위원회 "라 한다)를 둔다. ② 분쟁조정위원회는 위원장 1명을 포함한 30명 이내의 위원으로 구성하며, 위원은 당연직위원과 위촉위원으로 구 성한다.<개정 2015. 7. 24., 2023. 3. 14.> ③ 위촉위원은 다음 각 호의 어느 하나에 해당하는 사람 중에서 보호위원회 위원장이 위촉하고, 대통령령으로 정하 는 국가기관 소속 공무원은 당연직위원이 된다.<개정 2013. 3. 23., 2014. 11. 19., 2015. 7. 24.> 1. 개인정보 보호업무를 관장하는 중앙행정기관의 고위공무원단에 속하는 공무원으로 재직하였던 사람 또는 이에 상당하는 공공부문 및 관련 단체의 직에 재직하고 있거나 재직하였던 사람으로서 개인정보 보호업무의 경험이 있는 사람 법제처 27 국가법령정보센터 개인정보 보호법 2. 대학이나 공인된 연구기관에서 부교수 이상 또는 이에 상당하는 직에 재직하고 있거나 재직하였던 사람 3. 판사·검사 또는 변호사로 재직하고 있거나 재직하였던 사람 4. 개인정보 보호와 관련된 시민사회단체 또는 소비자단체로부터 추천을 받은 사람 5. 개인정보처리자로 구성된 사업자단체의 임원으로 재직하고 있거나 재직하였던 사람 ④ 위원장은 위원 중에서 공무원이 아닌 사람으로 보호위원회 위원장이 위촉한다.<개정 2013. 3. 23., 2014. 11. 19., 2015. 7. 24.> ⑤ 위원장과 위촉위원의 임기는 2년으로 하되, 1차에 한하

### Gemini Pro 실행 - Vertex AI Search as a Grounding Service

*   VertexAI API : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI

In [21]:
from langchain_google_vertexai.llms import VertexAI

gemini_pro = VertexAI( model_name = MODEL,
                  project=PROJECT_ID,
                  location=REGION,
                  verbose=True,
                  streaming=False,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 40
                 )

In [22]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain



prompt = PromptTemplate.from_template("""

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {context}
  Question : {question}

  """)

prompt = prompt.format(context=context,
                       question=question)

print(f"Prompt : {prompt}")

response = gemini_pro.invoke(prompt)
display(Markdown(response))

Prompt : 

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {'results-0': {'title': '개인정보 보호법(법률)(제19234호)(20240315)', 'uri': 'gs://it_laws_kr/law_pdf/개인정보 보호법(법률)(제19234호)(20240315).pdf', 'pageSpan': '27 ~ 41', 'content': '\n# 제7장 개인정보 분쟁조정위원회 <개정 2020. 2. 4.>\n제40조(설치 및 구성) ① 개인정보에 관한 분쟁의 조정(調停)을 위하여 개인정보 분쟁조정위원회(이하 “분쟁조정위원회 "라 한다)를 둔다. ② 분쟁조정위원회는 위원장 1명을 포함한 30명 이내의 위원으로 구성하며, 위원은 당연직위원과 위촉위원으로 구 성한다.<개정 2015. 7. 24., 2023. 3. 14.> ③ 위촉위원은 다음 각 호의 어느 하나에 해당하는 사람 중에서 보호위원회 위원장이 위촉하고, 대통령령으로 정하 는 국가기관 소속 공무원은 당연직위원이 된다.<개정 2013. 3. 23., 2014. 11. 19., 2015. 7. 24.> 1. 개인정보 보호업무를 관장하는 중앙행정기관의 고위공무원단에 속하는 공무원으로 재직하였던 사람 또는 이에 상당하는 공공부문 및 관련 단체의 직에 재직하고 있거나 재직하였던 사람으로서 개인정보 보호업무의 경험이 있는 사람 법제처 27 국가법령정보센터 개인정보 보호법 2. 대학이나 공인된 연구기관에서 부교수 이상 또는 이에 상당하는 직에 재직하고 있거나 재직하였던 사람 3. 판사·검사 또는 변호사로 재직하고 있거나 재직하였던 사람 4. 개인정보 보호와 관련된 시민사회단체 또는 소비자단체로부터 추천을 받은 사람 5. 개인정보처리자로 구성된 사업자단체의 임원으로 재직하고 있거나 재직하

**개인정보 보호법**

**목적:**
개인정보의 목적 외 수집, 오용, 남용, 무분별한 감시 및 추적 등으로 인한 폐해를 방지하고 인간의 존엄과 개인의 사생활을 보호하는 것.

**주요 내용:**

**개인정보의 정의:**
개인을 식별하거나 식별 가능하게 하는 정보로, 성명, 주민등록번호, 전화번호, 이메일 주소 등을 포함합니다.

**개인정보 처리의 원칙:**
* 목적 명시성: 개인정보는 명시된 목적을 달성하는 데 필요한 범위 내에서만 처리되어야 합니다.
* 동의: 개인정보를 처리하려면 정보주체의 동의가 필요합니다.
* 정확성 및 최신성: 개인정보는 정확하고 최신 상태로 유지되어야 합니다.
* 안전성: 개인정보는 무단 접근, 사용, 변경, 파괴로부터 안전하게 보호되어야 합니다.
* 보관 기간: 개인정보는 목적 달성에 필요한 기간 동안만 보관되어야 합니다.

**정보주체의 권리:**
* 개인정보 처리에 대한 정보 제공 권리
* 동의 여부 및 범위 선택 권리
* 개인정보 열람, 전송, 정정, 삭제, 파기 요구 권리
* 피해 구제 권리
* 완전히 자동화된 개인정보 처리에 대한 거부 또는 설명 요구 권리

**개인정보처리자의 의무:**
* 개인정보 처리 목적에 필요한 범위에서 개인정보의 정확성, 완전성, 최신성을 보장
* 개인정보를 안전하게 관리
* 개인정보 처리방침 등 개인정보 처리에 관한 사항 공개
* 정보주체의 권리를 보장

**개인정보 보호위원회:**
* 개인정보 보호에 관한 사무를 독립적으로 수행하는 기관
* 개인정보 침해요인 평가, 기본계획 및 시행계획 수립, 고충처리, 분쟁조정, 국제협력 등의 업무 수행

**제재:**
* 개인정보 보호법 위반에 대해 과징금, 시정조치, 고발, 징계권고 등의 제재가 부과될 수 있습니다.